In [32]:
import random
from collections import deque

# Define constants for suits and ranks
SUITS = ['hearts', 'diamonds', 'clubs', 'spades']
RANKS = list(range(2, 15))  # 6-10 (numeric), 11 (Jack), 12 (Queen), 13 (King), 14 (Ace)

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __repr__(self):
        return f"{self.rank} of {self.suit}"

class Environment:
    def __init__(self):
        self.deck = deque(self.shuffle_deck())
        self.discard_pile = []
        self.table_cards = []
        self.hands = [[], [], [], []]
        self.trump_card = None

    def shuffle_deck(self):
        deck = [Card(suit, rank) for suit in SUITS for rank in RANKS]
        random.shuffle(deck)
        return deck

    def deal_cards(self):
        for _ in range(6):
            for hand in self.hands:
                if self.deck:
                    hand.append(self.deck.popleft())
        if self.deck:
            self.trump_card = self.deck[-1]  # The last card in the deck is the trump

    def draw_card(self, player_id):
        if self.deck:
            self.hands[player_id].append(self.deck.popleft())

    def reset_table(self):
        self.discard_pile.extend(self.table_cards)
        self.table_cards = []

class Player:
    def __init__(self, strategy, player_id, team_id):
        self.strategy = strategy
        self.player_id = player_id
        self.team_id = team_id

    def choose_card(self, hand, table_cards, trump_suit):
        return self.strategy(hand, table_cards, trump_suit)

# Strategies

def greedy_strategy(hand, table_cards, trump_suit):
    # Play the weakest card possible
    non_trump_cards = [card for card in hand if card.suit != trump_suit]
    trump_cards = [card for card in hand if card.suit == trump_suit]
    if non_trump_cards:
        return min(non_trump_cards, key=lambda card: card.rank)
    return min(trump_cards, key=lambda card: card.rank) if trump_cards else None

def aggressive_strategy(hand, table_cards, trump_suit):
    # Play the strongest card possible
    non_trump_cards = [card for card in hand if card.suit != trump_suit]
    trump_cards = [card for card in hand if card.suit == trump_suit]
    if trump_cards:
        return max(trump_cards, key=lambda card: card.rank)
    return max(non_trump_cards, key=lambda card: card.rank) if non_trump_cards else None

class Game:
    def __init__(self):
        self.environment = Environment()
        self.players = [
            Player(greedy_strategy, 0, 0),
            Player(greedy_strategy, 1, 0),
            Player(aggressive_strategy, 2, 1),
            Player(aggressive_strategy, 3, 1),
        ]
        self.results = [0, 0]

    def play_round(self):
        env = self.environment
        env.reset_table()
        for player in self.players:
            card = player.choose_card(env.hands[player.player_id], env.table_cards, env.trump_card.suit)
            if card:
                env.table_cards.append(card)
                env.hands[player.player_id].remove(card)
        env.reset_table()

    def play_game(self):
        env = self.environment
        env.deal_cards()
        while any(env.hands):
            self.play_round()
        # Determine winning team (team with the fewest cards left wins)
        team_0_cards = sum(len(env.hands[i]) for i in [0, 1])
        team_1_cards = sum(len(env.hands[i]) for i in [2, 3])
        if team_0_cards < team_1_cards:
            self.results[0] += 1
        else:
            self.results[1] += 1

# Run experiments

def run_experiments(num_games=1000):
    results = [0, 0]
    for _ in range(num_games):
        game = Game()
        game.play_game()
        results[0] += game.results[0]
        results[1] += game.results[1]
    print(f"Greedy strategy wins: {results[0]}")
    print(f"Aggressive strategy wins: {results[1]}")

if __name__ == "__main__":
    run_experiments()


Greedy strategy wins: 0
Aggressive strategy wins: 1000
